In [20]:
#Part 1

import random

with open('20301027_Fathin Ishrak_CSE422_10_Lab_Assignment02_InputFile_Summer2024.txt', 'r') as file:
    data = file.readlines()
N, T = map(int, data[0].split())
courses = [line.strip() for line in data[1:]]

assert T >= N, "Number of timeslots must be greater than or equal to the number of courses."
chromosome_length = N * T

def fitness(chromosome):
    overlap_penalty = 0
    consistency_penalty = 0

    timeslot_segments = [chromosome[i*N:(i+1)*N] for i in range(T)]

    for segment in timeslot_segments:
        overlap_penalty += max(0, sum(segment) - 1)

    course_counts = [0] * N
    for segment in timeslot_segments:
        for i in range(N):
            if segment[i] == 1:
                course_counts[i] += 1

    consistency_penalty += sum(abs(count - 1) for count in course_counts)

    total_penalty = overlap_penalty + consistency_penalty
    return -total_penalty

def generate_initial_population(pop_size):
    population = []
    for _ in range(pop_size):
        chromosome = [0] * chromosome_length
        for i in range(N):
            timeslot = random.randint(0, T-1)
            chromosome[timeslot * N + i] = 1
        population.append(chromosome)
    return population

def select_parents(population, fitnesses):
    parents = []
    for _ in range(2):
        tournament = random.sample(list(zip(population, fitnesses)), 3)
        parents.append(max(tournament, key=lambda x: x[1])[0])
    return parents

def crossover(parent1, parent2):
    point = random.randint(1, chromosome_length - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(chromosome, mutation_rate):
    for i in range(chromosome_length):
        if random.random() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]

def genetic_algorithm(pop_size, mutation_rate, max_generations):
    population = generate_initial_population(pop_size)
    best_fitness = float('-inf')
    best_chromosome = None

    for generation in range(max_generations):
        fitnesses = [fitness(chrom) for chrom in population]
        new_population = []

        for _ in range(pop_size // 2):
            parent1, parent2 = select_parents(population, fitnesses)
            child1, child2 = crossover(parent1, parent2)
            mutate(child1, mutation_rate)
            mutate(child2, mutation_rate)
            new_population.extend([child1, child2])

        population = new_population
        fitnesses = [fitness(chrom) for chrom in population]
        current_best_fitness = max(fitnesses)
        current_best_chromosome = population[fitnesses.index(current_best_fitness)]

        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_chromosome = current_best_chromosome

    return best_chromosome, best_fitness

pop_size = 100
mutation_rate = 0.01
max_generations = 1000

best_solution, best_fitness = genetic_algorithm(pop_size, mutation_rate, max_generations)

with open('output1.txt', 'w') as file:
    file.write(''.join(map(str, best_solution)) + '\n')
    file.write(str(best_fitness) + '\n')
